# 1. Import dependencies

In [1]:
import copy
import cv2 as cv
import csv
import mediapipe as mp
import numpy as np
import itertools
import os

from collections import deque

# 2. Initialization

## a. Initialize variables and models

In [96]:
# Initialize Mediapipe's hand model 
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
hands = mp_hands.Hands(
    static_image_mode=True,   
    max_num_hands=1,
    min_detection_confidence=0.5,
    # min_tracking_confidence=0.5,
    model_complexity=1,
)

# Initialize main directory 
path = os.getcwd()
main_directory = os.path.dirname(path)

## b. Initialize functions

In [3]:
# Calculate landmark list 
def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_point = []

    # Keypoint
    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        # landmark_z = landmark.z

        landmark_point.append([landmark_x, landmark_y])

    return landmark_point

In [4]:
# Pre-processing calculated landmarks 
def pre_process_landmark(landmark_list):
    temp_landmark_list = copy.deepcopy(landmark_list)

    # Convert to relative coordinates
    base_x, base_y = 0, 0
    for index, landmark_point in enumerate(temp_landmark_list):
        if index == 0:
            base_x, base_y = landmark_point[0], landmark_point[1]

        temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
        temp_landmark_list[index][1] = base_y - temp_landmark_list[index][1]

    # Convert to a one-dimensional list
    temp_landmark_list = list(
        itertools.chain.from_iterable(temp_landmark_list))

    # Normalization
    max_value = max(list(map(abs, temp_landmark_list)))

    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list))

    return temp_landmark_list

## c. Initialize dataset template function

In [112]:
# Initialize number of hand keypoints and alphabet class
num_coords = 21
alphabet = "X"

In [113]:
# Initialize .csv file
def init_csv(feature):
    landmarks = ['class']
    for val in range(1, num_coords+1):
        landmarks += ['x{}'.format(val), 'y{}'.format(val)]

    with open(main_directory +'/dataset/03_csv_pre_combined/keypoints_{}.csv'.format(feature), mode='w', newline='') as f:
    # with open(main_directory +'/dataset/csv_pre_combined/keypoints_test.csv'.format(feature), mode='w', newline='') as f:
        csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerow(landmarks)

## d. Initialize dataset logging function

In [114]:
def logging_csv(feature, landmark_list):

    csv_path = main_directory + '/dataset/03_csv_pre_combined/keypoints_{}.csv'.format(feature)
    # csv_path = main_directory + '/dataset/keypoints_test.csv'
    
    with open(csv_path, 'a', newline="") as f:
        writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        writer.writerow([feature, *landmark_list])

    return

## e. Initialize image location

In [115]:
IMAGE_FILES = ['SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (1).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (2).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (3).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (4).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (5).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (6).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (7).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (8).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (9).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (10).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (11).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (12).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (13).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (14).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (15).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (16).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (17).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (18).jpg'.format(alphabet, alphabet),
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (19).jpg'.format(alphabet, alphabet),
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (20).jpg'.format(alphabet, alphabet),
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (21).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (22).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (23).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (24).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (25).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (26).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (27).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (28).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (29).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (30).jpg'.format(alphabet, alphabet),
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (31).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (32).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (33).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (34).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (35).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (36).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (37).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (38).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (39).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (40).jpg'.format(alphabet, alphabet),
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (41).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (42).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (43).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (44).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (45).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (46).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (47).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (48).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (49).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_V02/{}/{} (50).jpg'.format(alphabet, alphabet)]

# 3. Capture, re-normalize & write landmark keypoints into dataset

In [116]:
counter = 1

for idx, file in enumerate(IMAGE_FILES):
    # Read an image, flip it around y-axis for correct handedness output (see
    # above).
    image = cv.flip(cv.imread(file), 1)
    # Convert the BGR image to RGB before processing.
    results = hands.process(cv.cvtColor(image, cv.COLOR_BGR2RGB))
    
    if not results.multi_hand_landmarks:
        print("[[!]] Not detected : Image " + alphabet, counter)
        counter += 1
        continue
    
    image_height, image_width, _ = image.shape
    debug_image = copy.deepcopy(image)
    
    if results.multi_hand_landmarks is not None:
        for hand_landmarks in results.multi_hand_landmarks:

            # Convert pre-normalized landmark keys into pixels numbering
            landmark_list = calc_landmark_list(debug_image, hand_landmarks)
            
            # Convert into relative coordinates / normalize keys from wrist point
            pre_processed_landmark_list = pre_process_landmark(landmark_list)
            
            # Write/log received keypoints into dataset
            logging_csv(alphabet, pre_processed_landmark_list)
            print('Detected : Image '+ alphabet, counter)
            counter += 1
            # Visualize complete hand landmarks
            mp_drawing.draw_landmarks(
                debug_image,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())

Detected : Image X 1
Detected : Image X 2
Detected : Image X 3
Detected : Image X 4
Detected : Image X 5
Detected : Image X 6
Detected : Image X 7
Detected : Image X 8
Detected : Image X 9
Detected : Image X 10
Detected : Image X 11
Detected : Image X 12
Detected : Image X 13
Detected : Image X 14
Detected : Image X 15
Detected : Image X 16
Detected : Image X 17
Detected : Image X 18
Detected : Image X 19
Detected : Image X 20
[[!]] Not detected : Image X 21
[[!]] Not detected : Image X 22
[[!]] Not detected : Image X 23
[[!]] Not detected : Image X 24
[[!]] Not detected : Image X 25
Detected : Image X 26
Detected : Image X 27
Detected : Image X 28
Detected : Image X 29
Detected : Image X 30
Detected : Image X 31
Detected : Image X 32
Detected : Image X 33
Detected : Image X 34
Detected : Image X 35
Detected : Image X 36
[[!]] Not detected : Image X 37
[[!]] Not detected : Image X 38
Detected : Image X 39
[[!]] Not detected : Image X 40
Detected : Image X 41
Detected : Image X 42
Detec